# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

from jlib.transformer_char_predictor import TransformerCharPredictor
import jlib.data_utils as data_utils
import torch
import torch.nn as nn
import numpy as np
from torchprofile import profile_macs
text = ""
with open('data/sequence.txt', 'r') as f:
    text = f.read()


/home/jaskin/Intro_Deep_Learning/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# SeqLen 10

In [ ]:
seqlen = 10
def train_and_plot(seqlen: int):
    data = data_utils.gen_datasets(text, seqlen)
    train_data = data['train_dataset']
    val_data = data['val_dataset']
    alphabet: data_utils.Alphabet = data['alphabet']

    train_fetcher = data_utils.gen_data_loader(
        train_data,
        batch_size=32,
        workers = 6,
        cpu_prefetch= 20,
        gpu_prefetch=20
    )

    val_fetcher = data_utils.gen_data_loader(
        val_data,
        batch_size=len(val_data),
        workers = 6,
        cpu_prefetch= 10,
        gpu_prefetch=10
    )

    # model

    model = TransformerCharPredictor(
        alphabet_size = len(alphabet),
        max_len = seqlen,
        hidden_dim = 128,
        inner_dim = 2048,
        num_attn_heads = 2,
        num_attn_layers=3,
        cls_head_dims=[],
        dropout = 0.1
    )

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Model parameter count: {param_count:,}")

    test_input = next(iter(train_fetcher))[0]

    print(f"Model MACs: {profile_macs(model, test_input):,}")
    
# Model parameter count: 1,790,380
# Model MACs: 568,279,052



    model.train_model(
        epochs=100,
        train_fetcher=train_fetcher,
        val_fetcher=val_fetcher,
        optimizer = torch.optim.Adam,
        optimizer_kwargs={
            'lr': 3e-3,
            'betas': (0.9, 0.98),
            'eps': 1e-9,
            'weight_decay': 1e-5
        },
        min_accuracy=1,
        max_negative_diff_count=10,
        save_path=f'models/p1-{seqlen}.pth',
        stop_on_plateau=True,
    )

    fig = model.plot_training(f'Small Corpus, Sequence Length {seqlen}')
    fig.savefig(f'latex/images/p1-{seqlen}.png')
    
    del train_fetcher, val_fetcher, train_data, val_data, data, model, alphabet
    





In [3]:
train_and_plot(10)


Begin init data loader


Batch Size: 0.00244140625 MiB
Data Loader init time: 0.803153 s
Begin init fetcher
Fetcher init time: 0.877475 s
Begin init data loader
Batch Size: 0.03631591796875 MiB
Data Loader init time: 0.104730 s
Begin init fetcher
Fetcher init time: 0.169078 s
Model parameter count: 1,790,380
Model MACs: 568,279,052
Training TransformerCharPredictor

----------------------------------------------------------------------------------------------------------------------------------------------------


/home/jaskin/Intro_Deep_Learning/env/lib/python3.11/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unflatten". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/jaskin/Intro_Deep_Learning/env/lib/python3.11/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/jaskin/Intro_Deep_Learning/env/lib/python3.11/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scaled_dot_product_attention". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/jaskin/Intro_Deep_Learning/env/lib/python3.11/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


|       Epoch        |   Epoch Time (s)   |   Training Loss    |  Validation Loss   |Validation Accuracy |   Δ Accuracy (%)   |    Memory Usage    |
----------------------------------------------------------------------------------------------------------------------------------------------------
|         0          |      1.757319      |      2.874692      |      2.211866      |     44.222689      |      0.000000      |      0.163653      |
----------------------------------------------------------------------------------------------------------------------------------------------------
|         1          |      1.426046      |      1.906531      |      1.048573      |     78.571427      |     77.672211      |      0.167559      |
----------------------------------------------------------------------------------------------------------------------------------------------------
|         2          |      1.500337      |      1.243135      |      0.703344      |     84.726888      |

KeyboardInterrupt: 

In [ ]:
train_and_plot(20)


In [ ]:
train_and_plot(30)
